# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-02-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
from cassandra.cluster import Cluster

try: 
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)
    
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""CREATE KEYSPACE IF NOT EXISTS udacity
                    WITH REPLICATION = 
        { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




![](images/Capture.PNG)

there are the all attribute, data type for attribute and the selected attributed for each query to optimize the query not allow join in each query 

### Query 1

Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

`Table Name: song_info_by_session
column 1: artist           text
column 2: song             text
column 3: length           decimal
Column 4: sessionid        int
Column 5: itemlnsession    int
PRIMARY KEY(sessionid, itemlnsession)`

sessionid is a partition key and itemlnsession is cluster key

In [8]:
query = "create table if not exists song_info_by_session "
query = query + "(sessionid int, itemlnsession int, artist text, song text, length decimal, PRIMARY KEY (sessionid, itemlnsession))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

### read data from csv file and insert it in cassandra table

In [9]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into song_info_by_session (sessionid, itemlnsession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        try:
            session.execute(query, (int(line[8]),int(line[3]),line[0],line[9],float(line[5])))
        except Exception as e:
            print(e)

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
query = "select artist, length, song from song_info_by_session where sessionid=338 and itemlnsession=4"
try:
    rows=session.execute(query)
    for row in rows:
        print(row.artist,row.length,row.song)
except Exception as e:
    print(e)

Faithless 495.3073 Music Matters (Mark Knight Dub)


### Query 2

 Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

 `Table Name: song_playing_history_by_user
column 1: artist           text
column 2: song             text
column 3: first_name       text
column 4: last_name        text
Column 5: sessionid        int
Column 6: itemlnsession    int
Column 7: userid           int
PRIMARY KEY((userid,sessionid), itemlnsession)`

userid and sessionid are composed partition key and itemlnsession is cluster key (it used as cluster key to order the song order descending with itemlnsession)

In [11]:
query = "create table if not exists song_playing_history_by_user "
query = query + "(userid int,sessionid int,itemlnsession int,song text,artist text,first_name text,last_name text, PRIMARY KEY ((userid,sessionid), itemlnsession))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

### read data from csv file and insert it in cassandra table


In [12]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into song_playing_history_by_user (userid,sessionid,itemlnsession,song,artist,first_name,last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        try:
            session.execute(query, (int(line[10]),int(line[8]),int(line[3]),line[9],line[0],line[1],line[4]))
        except Exception as e:
            print(e)

#### Do a SELECT to verify that the data have been inserted into each table

In [13]:
query = "select artist, song, first_name, last_name from song_playing_history_by_user where userid = 10 and sessionid = 182 "
try:
    rows=session.execute(query)
    for row in rows:
        print(row.artist, row.song, row.first_name, row.last_name)
except Exception as e:
    print(e)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Query 3

Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

 `Table Name: who_listen_to_song
Column 1: userid           int
column 2: first_name       text
column 3: last_name        text
column 4: song             text
PRIMARY KEY( song ,user_id)`


song is partition key and userid is cluster key

In [14]:
query = "create table if not exists who_listen_to_song "
query = query + "(song text,userid int,first_name text,last_name text, PRIMARY KEY (song,userid))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

### read data from csv file and insert it in cassandra table


In [15]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into who_listen_to_song (song,userid, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        try:
            session.execute(query, (line[9],int(line[10]),line[1],line[4]))
        except Exception as e:
            print(e)

#### Do a SELECT to verify that the data have been inserted into each table

In [16]:
query = "select first_name , last_name, song from who_listen_to_song where song ='All Hands Against His Own'"
try:
    rows=session.execute(query)
    for row in rows:
        print(row.first_name, row.last_name,row.song)
except Exception as e:
    print(e)

Jacqueline Lynch All Hands Against His Own
Tegan Levine All Hands Against His Own
Sara Johnson All Hands Against His Own


### Drop the tables before closing out the sessions

In [17]:
query="drop table if exists Song_Details"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [18]:
query="drop table if exists user_history"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [19]:
query="drop table if exists who_listen_to_song"
try:
    session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()

# Thanks you